## Data Aggregations on Multi-Indices

In [1]:
import numpy as np
import pandas as pd
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                    names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                      names=['subject', 'type'])

# mock some data
np.random.seed(0) # stops reproducibility, same value is produced everytime
data1 = np.round(np.random.randn(4, 6), 1) # 1 represents decimals and rounds the no. to given no. of decimals
data1[:, ::2] *= 10
#data2=data1[:, ::2]*10
data =data1 + 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data
# This is actually a 4D data shown in a 2D dataFrame 

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      55.0  37.4  47.0  39.2  56.0  36.0
     2      47.0  36.8  36.0  37.4  38.0  38.5
2014 1      45.0  37.1  41.0  37.3  52.0  36.8
     2      40.0  36.1  11.0  37.7  46.0  36.3

In [4]:
data_mean=health_data.mean(level='year')
data_mean

subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
year                                        
2013     51.0  37.1  41.5  38.3  47.0  37.25
2014     42.5  36.6  26.0  37.5  49.0  36.55

In [5]:
data_mean1=health_data.mean(level='type',axis=1)
data_mean1

type               HR       Temp
year visit                      
2013 1      52.666667  37.533333
     2      40.333333  37.566667
2014 1      46.000000  37.066667
     2      32.333333  36.700000

### Combining Datasets: Concat and Append

Some of the most interesting studies of data come from combining different data
sources. These operations can involve anything from very straightforward concatenation 
of two different datasets, to more complicated database-style joins and merges
that correctly handle any overlaps between the datasets

 Series and DataFrames are
built with this type of operation in mind, and Pandas includes functions and methods
that make this sort of data wrangling fast and straightforward

In [16]:
def make_df(cols, ind):
   """Quickly make a DataFrame"""
   data = {c:[str(c) + str(i) for i in ind]
           for c in cols}
   return pd.DataFrame(data, ind)
# example DataFrame
make_df('ABC', range(3))


,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


### Simple Concatenation with pd.concat


Pandas has a function, pd.concat(), which has a similar syntax to np.concatenate
but contains a number of options that we’ll discuss momentarily

In [10]:
ser1 = pd.Series(['A', 'B', 'C'])#, index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'])#, index=[4, 5, 6])
pd.concat([ser1, ser2])


0    A
1    B
2    C
0    D
1    E
2    F
dtype: object

In [19]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])  # Both gives same output
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [21]:
df1=make_df('AB',['a','b'])
df2=make_df('CD',['a','b'])
print(df1);print(df2);print(pd.concat([df1,df2],axis=1))

    A   B
a  Aa  Ba
b  Ab  Bb
    C   D
a  Ca  Da
b  Cb  Db
    A   B   C   D
a  Aa  Ba  Ca  Da
b  Ab  Bb  Cb  Db


In [13]:
df1=make_df('AB',range(2))
df2=make_df('AB',range(2))
print(df1),print(df2),print(pd.concat([df1,df2]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A0  B0
1  A1  B1
    A   B
0  A0  B0
1  A1  B1
0  A0  B0
1  A1  B1


(None, None, None)

### Duplicate indices


One important difference between np.concatenate and pd.concat is that Pandas
concatenation preserves indices, even if the result will have duplicate indices!

In [17]:
x = make_df('AB', [0,1]) # pd.concat preserves indices means, even though we have asssigned new index to y, it behaves 
y = make_df('AB', [2,3])   # previous allotted indices, as concat func. indices are repeated
y.index=x.index
print(x),print(y),print(pd.concat([x,y]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


(None, None, None)

In [18]:
x1 = make_df('AB', ['a','b']) # pd.concat preserves indices means, even though we have asssigned new index to y, it behaves 
y1 = make_df('AB', ['c','d'])   # previous allotted indices, as concat func. indices are repeated
y.index=x.index
print(x),print(y),print(pd.concat([x,y]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


(None, None, None)

While this is valid within DataFrames, the
outcome is often undesirable. pd.concat() gives us a few ways to handle it.


>Catching the repeats as an error

If you’d like to simply verify that the indices in the
result of pd.concat() do not overlap, you can specify the verify_integrity flag

In [19]:
try:
 pd.concat([x, y], verify_integrity=True)
except ValueError as f:
 print("ValueError:", f)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


### Ignoring the index.

Sometimes the index itself does not matter, and you would prefer
it to simply be ignored. You can specify this option using the ignore_index flag. With
this set to True, the concatenation will create a new integer index for the resulting
Series

In [20]:
print(x);print(y);print(pd.concat([x,y],ignore_index=True)) # setting true, new index are created .

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [21]:
print(x1);print(y1);print(pd.concat([x1,y1],ignore_index=True)) # This is to tell , no matter what index is 
# It will be conerted to integers after using ignore_index=True

    A   B
a  Aa  Ba
b  Ab  Bb
    A   B
c  Ac  Bc
d  Ad  Bd
    A   B
0  Aa  Ba
1  Ab  Bb
2  Ac  Bc
3  Ad  Bd


### Adding MultiIndex keys

In [22]:
print(x);print(y);print(pd.concat([x,y],keys=['x','y'])) # creates multiIndex 

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


### Concatenation with joins

 In practice, data from different sources might have differ‐
ent sets of column names, and pd.concat offers several options in this case. Consider
the concatenation of the following two DataFrames,

In [23]:
df5 = make_df('ABC', [1, 2]) # here (join='outer') is the default value. Hence, union takes place
df6 = make_df('BCD', [3, 4])
print(df5); print(df6); print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


In [24]:
print(df5);print(df6);print(pd.concat([df5,df6],join='inner'))
# By default, the join is a union of the input columns (join='outer'), but we can change this to an 
# intersection of the columns using join='inner'

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


Now instead of join='inner', we would want only specified columns using func. df.columns

In [25]:
print(df5);print(df6);print(pd.concat([df5,df6],join_axes=[df5.columns])) #join_axes is not an arguement in this version

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4


TypeError: concat() got an unexpected keyword argument 'join_axes'

### The append() method

Because direct array concatenation is so common, Series and DataFrame objects
have an append method that can accomplish the same thing in fewer keystrokes

In [26]:
print(df5);print(df6);print(df5.append(df6))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


In [27]:
print(x1);print(y1);print(x1.append(y1))

    A   B
a  Aa  Ba
b  Ab  Bb
    A   B
c  Ac  Bc
d  Ad  Bd
    A   B
a  Aa  Ba
b  Ab  Bb
c  Ac  Bc
d  Ad  Bd


Keep in mind that unlike the append() and extend() methods of Python lists, the
append() method in Pandas does not modify the original object—instead, it creates a
new object with the combined data. Thus, if you plan to do multiple
append operations, it is generally better to build a list of DataFrames and pass them all
at once to the concat() function

### Combining Datasets: Merge and Join

## Categories of Joins

The pd.merge() function implements a number of types of joins: the one-to-one,
many-to-one, and many-to-many joins

### One-to-one joins


Perhaps the simplest type of merge expression is the one-to-one join, which is in
many ways very similar to the column-wise concatenation 

In [2]:
df1=pd.DataFrame({'employee':['bob','jake','lisa','sue'],
                  'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['lisa', 'bob', 'jake', 'sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1);print(df2)

  employee        group
0      bob   Accounting
1     jake  Engineering
2     lisa  Engineering
3      sue           HR
  employee  hire_date
0     lisa       2004
1      bob       2008
2     jake       2012
3      sue       2014


In [3]:
df0 =pd.concat([df1,df2])
df0

,employee,group,hire_date
0,bob,Accounting,NaN
1,jake,Engineering,NaN
2,lisa,Engineering,NaN
3,sue,HR,NaN
0,lisa,NaN,2004.0
1,bob,NaN,2008.0
2,jake,NaN,2012.0
3,sue,NaN,2014.0


In [6]:
#df1.merge(df2)
df3=pd.merge(df1,df2)
df3

,employee,group,hire_date
0,bob,Accounting,2008
1,jake,Engineering,2012
2,lisa,Engineering,2004
3,sue,HR,2014


The pd.merge() function recognizes that each DataFrame has an “employee” column,
and automatically joins using this column as a key

### Many-to-one joins


Many-to-one joins are joins in which one of the two key columns contains duplicate
entries. For the many-to-one case, the resulting DataFrame will preserve those 
duplicate entries as appropriate.

In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

  employee        group  hire_date
0      bob   Accounting       2008
1     jake  Engineering       2012
2     lisa  Engineering       2004
3      sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      bob   Accounting       2008      Carly
1     jake  Engineering       2012      Guido
2     lisa  Engineering       2004      Guido
3      sue           HR       2014      Steve


### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well
defined. If the key column in both the left and right array contains duplicates, then
the result is a many-to-many merge

In [8]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
print(df1);print(df5);print(pd.merge(df1,df5))

  employee        group
0      bob   Accounting
1     jake  Engineering
2     lisa  Engineering
3      sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      bob   Accounting          math
1      bob   Accounting  spreadsheets
2     jake  Engineering        coding
3     jake  Engineering         linux
4     lisa  Engineering        coding
5     lisa  Engineering         linux
6      sue           HR  spreadsheets
7      sue           HR  organization


These three types of joins can be used with other Pandas tools to implement a wide
array of functionality. But in practice, datasets are rarely as clean as the one we’re
working with here. In the following section, we’ll consider some of the options pro‐
vided by pd.merge() that enable you to tune how the join operations work.

### Specification of the Merge Key

#### The on keyword

Most simply, you can explicitly specify the name of the key column using the on 
keyword, which takes a column name or a list of column names

In [9]:
print(df1);print(df2),print(pd.merge(df1,df2,on='employee')) 
# The on key is used to specify a key column, but a column can be key column only if it is common in both dataframes.

  employee        group
0      bob   Accounting
1     jake  Engineering
2     lisa  Engineering
3      sue           HR
  employee  hire_date
0     lisa       2004
1      bob       2008
2     jake       2012
3      sue       2014
  employee        group  hire_date
0      bob   Accounting       2008
1     jake  Engineering       2012
2     lisa  Engineering       2004
3      sue           HR       2014


(None, None)

In [11]:
pd.merge(df1,df2,on='employee') # putting inside print function gives the output as above
# pd.merge(df1,df2) also gives same output as both have only one common column and it automatically becomes key without mentioning

,employee,group,hire_date
0,bob,Accounting,2008
1,jake,Engineering,2012
2,lisa,Engineering,2004
3,sue,HR,2014


#### The left_on and right_on keywords


At times you may wish to merge two datasets with different column names; for example, 
we may have a dataset in which the employee name is labeled as “name” rather
than “employee”

In [110]:
df3=pd.DataFrame({'name': ['bob','jake','lisa','sue'],
                  'salary':[50000,40000,45000,70000]})
print(df1);print(df3);print(pd.merge(df1,df3,left_on='employee',right_on='name'))
# if this left_on and right_on keywords are not used , we would not be able to perform pd.merge(df1,df3) as both don't have 
# a single common column name

  employee        group
0      bob   Accounting
1     jake  Engineering
2     lisa  Engineering
3      sue           HR
   name  salary
0   bob   50000
1  jake   40000
2  lisa   45000
3   sue   70000
  employee        group  name  salary
0      bob   Accounting   bob   50000
1     jake  Engineering  jake   40000
2     lisa  Engineering  lisa   45000
3      sue           HR   sue   70000


Now , we have a redundant(repeated) column that we don't need any more and we can delete that using delete keyword

In [92]:
print(pd.merge(df1,df3,left_on='employee',right_on='name').drop('name',axis=1))
# axis is mandatory to be mentioned as default axis is 0 and so the drop() function would have searched for axis=0.

  employee        group  salary
0      bob   Accounting   50000
1     jake  Engineering   40000
2     lisa  Engineering   45000
3      sue           HR   70000


#### The left_index and right_index keywords

Sometimes, rather than merging on a column, you would instead like to merge on an
index. For example, your data might look like this


In [13]:
df1a=df1.set_index('employee')
df2a=df2.set_index('employee')
print(df1a);print(df2a)

                group
employee             
bob        Accounting
jake      Engineering
lisa      Engineering
sue                HR
          hire_date
employee           
lisa           2004
bob            2008
jake           2012
sue            2014


u can merge by specifying true in left_index and right_index

In [14]:
print(pd.merge(df1a,df2a,left_index=True,right_index=True)) # It is used to merge as index as common

                group  hire_date
employee                        
bob        Accounting       2008
jake      Engineering       2012
lisa      Engineering       2004
sue                HR       2014


In [15]:
df1a.reset_index() # if i use reset_index ,previous column turned index would turn back to column

,employee,group
0,bob,Accounting
1,jake,Engineering
2,lisa,Engineering
3,sue,HR


In [16]:
x=pd.merge(df1a,df2a,left_index=True,right_index=True)

In [17]:
x.reset_index()

,employee,group,hire_date
0,bob,Accounting,2008
1,jake,Engineering,2012
2,lisa,Engineering,2004
3,sue,HR,2014


For convenience, DataFrames implement the join() method, which performs a
merge that defaults to joining on indices

In [23]:
 print(df1a); print(df2a); print(df1a.join(df2a)) # when both have some column as index rather than natural integers

                group
employee             
bob        Accounting
jake      Engineering
lisa      Engineering
sue                HR
          hire_date
employee           
lisa           2004
bob            2008
jake           2012
sue            2014
                group  hire_date
employee                        
bob        Accounting       2008
jake      Engineering       2012
lisa      Engineering       2004
sue                HR       2014


In [112]:
print(df1a);print(df3);print(pd.merge(df1a,df3,left_index=True,right_on='name'))

                group
employee             
bob        Accounting
jake      Engineering
lisa      Engineering
sue                HR
   name  salary
0   bob   50000
1  jake   40000
2  lisa   45000
3   sue   70000
         group  name  salary
0   Accounting   bob   50000
1  Engineering  jake   40000
2  Engineering  lisa   45000
3           HR   sue   70000


### Specifying Set Arithmetic for Joins


In [22]:
df1a.join?

In [24]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                    columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                    columns=['name', 'drink'])
print(df6);print(df7);print(pd.merge(df6,df7,)) # both dataframes have only one 'name' in common.

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


In [29]:
pd.merge(df6,df7,how='outer') # above, how is set default to inner, by specifying outer , this result is obtained

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [27]:
pd.merge(df6,df7,how='right') 

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


In [28]:
pd.merge(df6,df7,how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


### Overlapping Column Names: The suffixes Keyword


In [32]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
print(df8); print(df9); print(pd.merge(df8,df9,on='name')) # merges keeping name as key and assigning suffix to ranks 

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [34]:
print(pd.merge(df8,df9)) # This gives an empty dataframe as ranks in both dataframes are differently assigned

Empty DataFrame
Columns: [name, rank]
Index: []


In [36]:
# I could have mentioned the suffixes by myself as depicted below
print(pd.merge(df8,df9,on='name',suffixes=['_L','_R']))

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


These suffixes work in any of the possible join patterns, and work also if there are
multiple overlapping columns.

### Example: US States Data